In [1]:
import xarray as xr
import numpy as np
import glob as glob
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from matplotlib.animation import FuncAnimation
import imageio.v2 as imageio
import numpy as np
import sparse

In [2]:
#file1 = '/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_1008_withrestart/lnd/NSSP534frc2esm_f19_tn14_1008_withrestart.clm2.h1.2080_all_ncr.nc'      #default
#file1 = '/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0908_withrestart_withmodlanduse2/lnd/NSSP534frc2esm_f19_tn14_0908_withrestart_withmodlanduse2.clm2.h1.2080_all_nrc.nc'   #corn
file1 = '/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0509_sugarcane/lnd/NSSP534frc2esm_f19_tn14_0509_sugarcane.clm2.h1.2080_all.nc'   #sugarcane
data = xr.open_dataset(file1, decode_times=False)
#data2 = xr.open_dataset(file2, decode_times=False)
for var_name in data.variables:
        if data[var_name].dtype == 'int32':
            data[var_name] = data[var_name].astype('float64')

In [3]:
pft_constants = xr.open_dataset("/cluster/shared/noresm/inputdata/lnd/clm2/paramdata/clm5_params.c171117.nc")
pftnames = pft_constants.pftname

In [4]:
def to_sparse(data, vegtype, jxy, ixy, shape):
    """
    Takes an input numpy array and converts it to a sparse array.

    Parameters
    ----------
    data: numpy.ndarray
        1D or 2D Data stored in compressed form.
    vegtype: numpy.ndarray

    jxy: numpy.ndarray
        Latitude index
    ixy: numpy.ndarray
        Longitude index
    shape: tuple
        Shape provided as sizes of (vegtype, jxy, ixy) in uncompressed
        form.

    Returns
    -------
    sparse.COO
        Sparse nD array
    """
    import sparse

    # This constructs a list of coordinate locations at which data exists
    # it works for arbitrary number of dimensions but assumes that the last dimension
    # is the "stacked" dimension i.e. "pft"
    if data.ndim == 1:
        coords = np.stack([vegtype, jxy - 1, ixy - 1], axis=0)
    elif data.ndim == 2:
        # generate some repeated time indexes
        # [0 0 0 ... 1 1 1... ]
        itime = np.repeat(np.arange(data.shape[0]), data.shape[1])
        # expand vegtype and friends for all time instants
        # by sequentially concatenating each array for each time instants
        tostack = [np.concatenate([array] * data.shape[0]) for array in [vegtype, jxy - 1, ixy - 1]]
        coords = np.stack([itime] + tostack, axis=0)
    else:
        raise NotImplementedError

    return sparse.COO(
        coords=coords,
        data=data.ravel(),
        shape=data.shape[:-1] + shape,
        fill_value=np.nan,
    )


def convert_pft_variables_to_sparse(dataset, pftnames):
    """
    Convert 2D PFT variables in dataset to 4D sparse arrays.

    Parameters
    ----------
    dataset: xarray.Dataset
        Dataset with DataArrays that have a `pft` dimension.

    Returns
    -------
    xarray.Dataset
        Dataset whose "PFT" arrays are now sparse arrays
        with `pft` dimension expanded out to (type, lat, lon)
    """

    import sparse
    import xarray as xr

    # extract PFT variables
    pfts = xr.Dataset({k: v for k, v in dataset.items() if "pft" in v.dims})

    # extract coordinate index locations
    ixy = dataset.pfts1d_ixy.astype(int)
    jxy = dataset.pfts1d_jxy.astype(int)
    vegtype = dataset.pfts1d_itype_veg.astype(int)
    npft = len(pftnames.data)

    # expected shape of sparse arrays to pass to `to_sparse` (excludes time)
    output_sizes = {
        "vegtype": npft,
        "lat": dataset.sizes["lat"],
        "lon": dataset.sizes["lon"],
    }

    result = xr.Dataset()
    # we loop over variables so we can specify the appropriate dtype
    for var in pfts:
        result[var] = xr.apply_ufunc(
            to_sparse,
            pfts[var],
            vegtype,
            jxy,
            ixy,
            kwargs=dict(shape=tuple(output_sizes.values())),
            input_core_dims=[["pft"]] * 4,
            output_core_dims=[["vegtype", "lat", "lon"]],
            dask="parallelized",
            dask_gufunc_kwargs=dict(
                meta=sparse.COO(np.array([], dtype=pfts[var].dtype)),
                output_sizes=output_sizes,
            ),
            keep_attrs=True,
        )

    # copy over coordinate variables lat, lon
    result = result.update(dataset[["lat", "lon"]])
    result["vegtype"] = pftnames.data
    # save the dataset attributes
    result.attrs = dataset.attrs
    return result

In [5]:
sparse_data = convert_pft_variables_to_sparse(data, pftnames)

In [18]:
sparse_data
sparse_data.NPP

Format,coo
Data Type,float64
Shape,"(12, 79, 96, 144)"
nnz,2394420
Density,0.18270829670651664
Read-only,True
Size,91.3M
Storage ratio,0.9


In [17]:
gpp = sparse_data.NPP.compute()
gpp = gpp.copy(data=gpp.data.todense())
gpp

<xarray.DataArray 'NPP' (time: 12, vegtype: 79, lat: 96, lon: 144)>
array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
...
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]]])
Coordinates:
  * time     (time) float64 31.0 59.0 90.0 120.0 ... 273.0 304.0 334.0 365.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * vegtype  (vegtype) |S40 b'not_vegetated                           ' ... b...
Attributes:
    long_name:     net primary production
    units:         gC/m^2/s
    cell_methods:  time: mean

In [7]:
gpp.to_netcdf("/cluster/projects/nn9576k/anusha/DATA/NSSP534frc2esm_f19_tn14_1008_sugarcane_PFTleveldata.nc")

In [28]:
subset = data.NPP.isel(time=0).load()
subset
ixy = data.pfts1d_ixy.load().astype(int)
jxy = data.pfts1d_jxy.load().astype(int)
vegtype = data.pfts1d_itype_veg.load().astype(int)
vegtype.data

array([0, 1, 2, ..., 0, 0, 0])

In [22]:
eye = sparse.COO(coords=[[0, 1, 2], [0, 1, 2]], data=[1, 1, 1])
eye
eye.todense() # identity matrix!
array = sparse.COO(
    coords=[[0, 1, 2], [0, 1, 2]],
    data=np.array([1, 1, 1], dtype=np.float32),
    shape=(4, 4),
    fill_value=np.nan,
)
array.todense()


ixy
coords = np.stack([vegtype, jxy - 1, ixy - 1], axis=0)
coords.shape

(3, 222182)

In [23]:
data

<xarray.Dataset>
Dimensions:             (levgrnd: 25, levlak: 10, levdcmp: 25, time: 12,
                         hist_interval: 2, lon: 144, lat: 96, gridcell: 5701,
                         landunit: 17795, column: 142368, pft: 222182)
Coordinates:
  * levgrnd             (levgrnd) float64 0.01 0.04 0.09 ... 19.48 28.87 42.0
  * levlak              (levlak) float64 0.05 0.6 2.1 4.6 ... 25.6 34.33 44.77
  * levdcmp             (levdcmp) float64 0.01 0.04 0.09 ... 19.48 28.87 42.0
  * time                (time) float64 31.0 59.0 90.0 ... 304.0 334.0 365.0
  * lon                 (lon) float64 0.0 2.5 5.0 7.5 ... 352.5 355.0 357.5
  * lat                 (lat) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
Dimensions without coordinates: hist_interval, gridcell, landunit, column, pft
Data variables: (12/87)
    mcdate              (time) float64 2.08e+07 2.08e+07 ... 2.08e+07 2.081e+07
    mcsec               (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    mdcur               (time) float64 31.0 59.0 90.0 ... 304.0 334.0 365.0
    mscur               (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    nstep               (time) float64 1.488e+03 2.832e+03 ... 1.752e+04
    time_bounds         (time, hist_interval) float64 ...
    ...                  ...
    TOTECOSYSN          (time, column) float64 ...
    TOTSOMC             (time, column) float64 ...
    TOTSOMN             (time, column) float64 ...
    TOTVEGC             (time, pft) float64 ...
    TOTVEGN             (time, pft) float64 ...
    WOODC               (time, pft) float64 ...
Attributes: (12/103)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   Fri Aug 25 11:22:53 2023: ncrc...
    source:                                    Community Land Model CLM4.0
    hostname:                                  betzy
    ...                                        ...
    cft_tropical_soybean:                      63
    cft_irrigated_tropical_soybean:            64
    time_period_freq:                          month_1
    Time_constant_3Dvars_filename:             ./NSSP534frc2esm_f19_tn14_0908...
    Time_constant_3Dvars:                      ZSOI:DZSOI:WATSAT:SUCSAT:BSW:H...
    NCO:                                       netCDF Operators version 5.0.3...

In [26]:
sparse.COO(coords=coords, data=subset.data)

ValueError: data must be a scalar or 1-dimensional.